In [67]:
import random
import cv2
from ultralytics import YOLO
import time
from pymycobot import MyCobot

# for mypalletizer
from pymycobot import MyPalletizer

# for myBuddy
from pymycobot import MyBuddy
from pymycobot import MyCobot
import time

In [65]:
mc = MyCobot("COM5",115200)
mc.release_all_servos()

In [66]:
mc.set_basic_output(2, 1)
mc.set_basic_output(5, 1)

[]

In [48]:
grid_positions_angles = [
    [5.71, -32.51, -141.24, 59.85, 2.37, 155.39],
    [20.83, -31.9, -141.24, 60.73, 13.62, 164.09], 
    [40.69, -35.59, -128.4, 51.5, 14.06, 165.32],
    [-0.87, -43.85, -128.05, 75.23, 5.88, 156.88],
    [18.28, -41.13, -128.4, 70.4, 7.64, 169.36],
    [37.61, -47.54, -127.79, 81.21, 9.66, 172.7],
    [2.9, -52.38, -105.11, 65.21, 1.84, 170.77],
    [15.2, -52.64, -106.61, 67.85, 6.5, 175.95],
    [31.64, -53.7, -104.58, 66.88, 2.28, -177.36]
    # ... (repeat for all grid positions)
    ]

In [49]:
# mc.send_angles(grid_positions_angles[3 * 0 + 1],50)


In [68]:
def print_board(board):
    for row in board:
        print(" | ".join(row))
        print("-" * 5)

def check_winner(board, player):
    # Check rows, columns, and diagonals for a win
    for i in range(3):
        if all(board[i][j] == player for j in range(3)) or all(board[j][i] == player for j in range(3)):
            return True
    if all(board[i][i] == player for i in range(3)) or all(board[i][2 - i] == player for i in range(3)):
        return True
    return False

def is_board_full(board):
    return all(board[i][j] != ' ' for i in range(3) for j in range(3))

def get_empty_cells(board):
    return [(i, j) for i in range(3) for j in range(3) if board[i][j] == ' ']

def player_move(board):
    while True:
        try:
            row = int(input("Enter the row (0, 1, or 2): "))
            col = int(input("Enter the column (0, 1, or 2): "))
            if board[row][col] == ' ':
                return row, col
            else:
                print("Cell already occupied. Try again.")
        except (ValueError, IndexError):
            print("Invalid input. Please enter a number between 0 and 2.")

def minimax(board, depth, maximizing_player):
    if check_winner(board, 'O'):
        return 1
    elif check_winner(board, 'X'):
        return -1
    elif is_board_full(board):
        return 0

    if maximizing_player:
        max_eval = float('-inf')
        for i, j in get_empty_cells(board):
            board[i][j] = 'O'
            eval = minimax(board, depth + 1, False)
            board[i][j] = ' '
            max_eval = max(max_eval, eval)
        return max_eval
    else:
        min_eval = float('inf')
        for i, j in get_empty_cells(board):
            board[i][j] = 'X'
            eval = minimax(board, depth + 1, True)
            board[i][j] = ' '
            min_eval = min(min_eval, eval)
        return min_eval

def best_move(board):
    best_val = float('-inf')
    best_move = None
    for i, j in get_empty_cells(board):
        board[i][j] = 'O'
        move_val = minimax(board, 0, False)
        board[i][j] = ' '
        if move_val > best_val:
            best_move = (i, j)
            best_val = move_val
    return best_move


In [69]:
def crop_image(image, coordinates):
    x, y, w, h = coordinates
    cropped_image = image[y:y+h, x:x+w]
    return cropped_image

def get_object_in_region(region_image, model,i,j):
    results = model(region_image, save=False)
    if results[0]:
        print(i)
        print(j)
        print("here")
        print(results[0].names[results[0].boxes.cls.numpy()[0]])
        return results[0].names[results[0].boxes.cls.numpy()[0]]
    return None


In [70]:
def update_board_from_image(board, model,coors):
    # Load the image
#     image = cv2.imread(image_path)
    cap = cv2.VideoCapture(0)

    # Automatically set the initial cropping region for subsequent frames
    crop_x, crop_y, crop_w, crop_h = coors
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        cropped_image = frame[crop_y:crop_y+crop_h, crop_x:crop_x+crop_w]
        cv2.imshow("Cropped Region", cropped_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        height, width, _ = cropped_image.shape
        region_height = height // 3
        region_width = width // 3

        for i in range(3):
            for j in range(3):
                region = cropped_image[i * region_height: (i + 1) * region_height,
                                   j * region_width: (j + 1) * region_width]

            # Get the object in the region using YOLO
                detected_object = get_object_in_region(region, model,i,j)
   
                cv2.imshow("Region",region)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
            # Update the board based on the detected object
                if detected_object == 'teddy bear':
                    board[i][j] = 'X'
                elif detected_object == 'bottle':
                    board[i][j] = 'O'

    # Display the image with rectangles around detected regions
        cv2.imshow("Detected Regions", cropped_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        break
    return board



In [74]:
def play_tic_tac_toe():
    yolo_model = YOLO('yolov8n.pt')
    grid_positions_angles = [
    [5.71, -32.51, -141.24, 59.85, 2.37, 155.39],
    [20.83, -31.9, -141.24, 60.73, 13.62, 164.09], 
    [40.69, -35.59, -128.4, 51.5, 14.06, 165.32],
    [-0.87, -43.85, -128.05, 75.23, 5.88, 156.88],
    [18.28, -41.13, -128.4, 70.4, 7.64, 169.36],
    [37.61, -47.54, -127.79, 81.21, 9.66, 172.7],
    [2.9, -52.38, -105.11, 65.21, 1.84, 170.77],
    [15.2, -52.64, -106.61, 67.85, 6.5, 175.95],
    [31.64, -53.7, -104.58, 66.88, 2.28, -177.36]
    # ... (repeat for all grid positions)
    ]
    board = [[' ' for _ in range(3)] for _ in range(3)]
    player_symbol = 'X'
    computer_symbol = 'O'
    mc = MyCobot("COM5",115200)
    mc.release_all_servos()
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    coordinates = cv2.selectROI("Select Region", frame, False, False)
    cv2.destroyAllWindows()
    initial_cropped_frame = crop_image(frame, coordinates)
    cv2.imshow("Init Crop", initial_cropped_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cap.release()
    
    print("Welcome to Tic-Tac-Toe!")
    mc.send_angles([0,0,0,0,0,0],50)
    while True:
        
        print("Start of round board")
        print_board(board)
        # Player move
        time.sleep(5)
        input("Press Enter when player has placed a sticker...")
        board=update_board_from_image(board,yolo_model,coordinates)
        print("After Player Move")
        print_board(board)
#         player_row, player_col = player_move(board)
#         board[player_row][player_col] = player_symbol

        # Check if the player wins
        if check_winner(board, player_symbol):
            print_board(board)
            print("Congratulations! You win!")
            break

        # Check if the board is full (tie)
        if is_board_full(board):
            print_board(board)
            print("It's a tie!")
            break

        # Computer move
        print("Computer's turn:")
        computer_row, computer_col = best_move(board)
        board[computer_row][computer_col] = computer_symbol
        
        #Pump On
        mc.set_basic_output(2, 0)
        mc.set_basic_output(5, 0)
        time.sleep(8)
        
        #Move arm to correct grid location and Place a sticker on the grid
        mc.send_angles(grid_positions_angles[3 * computer_row + computer_col],50)

        # Pump Off
        time.sleep(5)
        mc.set_basic_output(2, 1)
        mc.set_basic_output(5, 1)
        mc.send_angles([0,0,0,0,0,0,0],50)
        time.sleep(5)
        # Check if the computer wins
        if check_winner(board, computer_symbol):
            print_board(board)
            print("Computer wins! Better luck next time.")
            break

        # Check if the board is full (tie)
        if is_board_full(board):
            print_board(board)
            print("It's a tie!")
            break




In [80]:
if __name__ == "__main__":
    # Load YOLO model
    
    play_tic_tac_toe()

Welcome to Tic-Tac-Toe!
Start of round board
  |   |  
-----
  |   |  
-----
  |   |  
-----
Press Enter when player has placed a sticker...

0: 576x640 (no detections), 183.6ms
Speed: 8.0ms preprocess, 183.6ms inference, 8.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 178.9ms
Speed: 0.0ms preprocess, 178.9ms inference, 2.5ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 1 tv, 172.8ms
Speed: 10.6ms preprocess, 172.8ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)
0
2
here
tv

0: 576x640 (no detections), 174.2ms
Speed: 8.0ms preprocess, 174.2ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 157.8ms
Speed: 8.0ms preprocess, 157.8ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 157.7ms
Speed: 8.0ms preprocess, 157.7ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 166.0ms
Speed: 8.0